In [179]:
import sys
print(sys.executable)
!{sys.executable} -m pip install gurobipy
from gurobipy import *
import math

/Users/Abe/opt/anaconda3/bin/python


In [180]:
#open jld
#https://docs.h5py.org/en/stable/high/group.html
#https://docs.h5py.org/en/stable/high/attr.html
import h5py
x1_data = h5py.File("X1.jld", "r")
x2_data = h5py.File("X2.jld", "r")
x3_data = h5py.File("X3.jld", "r")

#print(x1_data.keys(),x2_data.keys(),x3_data.keys())

# f["input"].value, f["output"].value
# f["X"].value
# f["_creator"]

print(x1_data["X"])
print(x1_data["X"][0])
print(x1_data["X"][1])

#x1_data["_creator"].keys()

<HDF5 dataset "X": shape (2, 25), type "<f8">
[-9.31431352 -5.66901261 -5.58994131 -6.48529461 -4.64803558 -6.92318026
 -4.9219469  -6.18895012 -6.383588   -4.83052083 -5.54884829 -6.03902456
 -9.36696392 -1.01326107  0.08758034  1.34522284  1.02763538  0.18065355
  0.96876053  2.14521255 -3.81989054 -4.94544952 -3.29144358 -2.36855237
 -4.05178677]
[ 21.85248196  22.82571229  21.70189592  20.44812788 -13.57047711
 -13.45455931 -13.16898973 -15.16087729 -14.11121926 -13.3698081
 -13.74643136 -13.17614957 -13.63194927  -5.99053048  -4.86670161
  -5.53625377  -6.47446421  -4.69903272  -4.01117971  -6.54907345
   1.50498652   1.28581208   2.7638501    2.11314414   1.67859478]


In [181]:
instance = 3
# 1, 2, or 3

if instance == 1:
    n_min = 2
    n_max = 8
    d_max = 10
    X = [x1_data["X"][0], x1_data["X"][1]]
    
elif instance == 2:
    n_min = 4
    n_max = 6
    d_max = 9
    X = [x2_data["X"][0], x2_data["X"][1]]
    
elif instance == 3:

    n_min = 4
    n_max = 6
    d_max = 9 

    X = [x3_data["X"][0], x3_data["X"][1]]

else:
    print("Error: instance needs to be 1, 2, or 3")

n = len(x1_data["X"][0]) #should be 25

In [182]:
# create model
myModel = Model( "ClientDepot_MinClusters" )

In [183]:
# decision variables and parameters

# Y is an assignment matrix:
# Y[i][j] = 1: client i is assigned to depot j, otherwise 0
# each entry of Y is its own decision variable
Y = []
for i in range(n):
    Y.append([])
    for j in range(n):
        Y[i].append(0)
        Y[i][j] = myModel.addVar(lb = 0, ub = 1.0, vtype = GRB.BINARY )


#depots is a vector that indicates whether a depot has clients assigned to it
#depots[j] = 0: no clients assigned to depot j.
#          = 1: one or more clients assigned to depot j.
depots = [myModel.addVar(lb = 0, ub = 1.0, vtype = GRB.BINARY ) for j in range(n)]

#parallel (perpendicular?) list to depots, used just for summing 
clients = [None for j in range(n)] 

# Z is a matrix that indicates whether clients i and ii are in the same depot
Z = []
for i in range(len(clients)):
    Z.append([])
    for ii in range(len(clients)):
        Z[i].append(None)
        Z[i][ii] = myModel.addVar(lb = 0, ub = 1.0, vtype = GRB.BINARY )


#Y = myModel.addVar( vtype = GRB.BINARY, name = "Y" )
myModel.update()
#print(Y)
        

In [184]:
# objective: MINIMIZE NUMBER OF CLUSTERS (part a)

objExpr = LinExpr()

for j in range(len(depots)):
    objExpr += depots[j]

myModel.setObjective( objExpr , GRB.MINIMIZE )

print(objExpr)

<gurobi.LinExpr: C625 + C626 + C627 + C628 + C629 + C630 + C631 + C632 + C633 + C634 + C635 + C636 + C637 + C638 + C639 + C640 + C641 + C642 + C643 + C644 + C645 + C646 + C647 + C648 + C649>


In [185]:
# create expressions for constraints and add to the model

# FIRST CONSTRAINT: definition of "depots" 
#(vector that indicates if a column of Y represents a depot)

firstConst = [LinExpr() for j in depots]
for j in range(len(depots)):
    
    for i in range(len(clients)):
        firstConst[j] += Y[i][j]/(n+1)
    
    # 0 < firstConst[j] < 1 if there is at least 1 client assigned to depot j
        
    myModel.addConstr( lhs = depots[j] , sense = GRB.GREATER_EQUAL , rhs = firstConst[j] )
    #if there are some clients assigned to depot j (nonzero entries in the j-th col of Y)
    #then depots[j] must be non-zero. since depots is binary, it will take on the value 1.
    
    myModel.addConstr( lhs = depots[j], sense = GRB.LESS_EQUAL, rhs = firstConst[j]*(n+1) + 0.5 )
    # if there is no client assigned to depots[j], then the rhs will be 0.5, so
    # depots[j] will be forced to be less than 0.5, and the only option is zero
    # NOTE that this contraint is automatically satisfied if firstConst[j] is nonzero since the lhs > 1.
    
    
    
#SECOND CONSTRAINT: client-depot uniqueness
#ensures that the sum of each row of Y is exactly 1 (each client assigned to one unique depot)

secondConst = [LinExpr() for i in clients]
for i in range(len(clients)):
    
    for j in range(len(depots)):
        secondConst[i] += Y[i][j]
    
    myModel.addConstr( lhs = secondConst[i] , sense = GRB.EQUAL , rhs = 1 )

    
#THIRD CONSTRAINT: min and max clients constraint

thirdConst = [LinExpr() for j in depots]
for j in range(len(depots)):
    
    for i in range(len(clients)):
        thirdConst[j] += Y[i][j]
    
    myModel.addConstr( lhs = thirdConst[j] , sense = GRB.LESS_EQUAL , rhs = n_max*depots[j] )
    myModel.addConstr( lhs = thirdConst[j] , sense = GRB.GREATER_EQUAL , rhs = n_min*depots[j] )
    
# FOURTH CONSTRAINT: max distance constraint

for j in range(len(depots)):
    
    #compare the distances between all clients in the column
    
    for i in range(len(depots)):
        for ii in range(len(depots)): #TRIPLE LOOP :)
            
            d = math.sqrt((X[0][i] - X[0][ii])**2 + (X[1][i] - X[1][ii])**2)
            act = 10000*(1 - Y[i][j]) + 10000*(1 - Y[ii][j])
            #if Y[i][j] or Y[ii][j] = 0, the constraint will be satisfied automatically
            #if they are both 1, then the constraint is "act"ivated, 
            #so the distance between that pair must be less than d_max
            
            myModel.addConstr( lhs = d , sense = GRB.LESS_EQUAL , rhs = d_max + act )

In [186]:
# integrate objective and constraints into the model
myModel.update()
# write the model in a file to make sure it is constructed correctly
myModel.write( filename = "testOutput_A.lp" )
# optimize the model
myModel.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15750 rows, 1275 columns and 33850 nonzeros
Model fingerprint: 0xd2480bea
Variable types: 0 continuous, 1275 integer (1275 binary)
Coefficient statistics:
  Matrix range     [4e-02, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+04]
Presolve removed 13196 rows and 644 columns
Presolve time: 0.07s

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.12 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [187]:
# print optimal objective and optimal solution
print( "\nOptimal Objective: " + str( myModel.ObjVal ) )

print( "\nOptimal Solution:" )
Y_display = []
for i in range(len(clients)):
    Y_display.append([])
    for j in range(len(depots)):
        Y_display[i].append(int(Y[i][j].x))

for i in range(len(clients)):
    print(Y_display[i])

AttributeError: Unable to retrieve attribute 'ObjVal'